# High-Performance Pandas: eval() and query()

## Motivating `query()` and `eval()`: Compound Expression

In [3]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

4.29 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

221 ms ± 4.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

In [11]:
%timeit df1 + df2 + df3 + df4

94.1 ms ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit pd.eval('df1 + df2 + df3 + df4')

46.6 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The `eval()` version of this expression is about 50% faster (and uses much less memory), while giving the same result:

### Arithmetic operators

`pd.eval()` supports all arithmetic operators. For example:

In [14]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

### Comparison operators

`pd.eval()` supports all comparison operators, including chained expression:

In [15]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

### Bitwise operators

`pd.eval()` supports the `&` and `|` bitwise operators:

In [16]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [17]:
# Also supports boolean
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

### Object attributes and indices

`pd.eval()` supports access to objects attributes via the `obj.attr` syntax, and indexes via the `obj[index]` syntax:

In [18]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

In [19]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [20]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

In [23]:
# Cool thing about using eval()
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

In [24]:
# Calculate and store the result in new column
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [25]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


## Local variables in DataFrame.eval()

The `DataFame.eval()` method supports an addition syntax that lets it work with Python variables. Consider the following:

In [26]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

# DataFrame.query() Method

The `DataFrame` has another method based on evaluated strings, called the `query()` method. Consider the following:

In [27]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

In [28]:
# Above using Query
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [29]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

In [31]:
# size of our array
df.values.nbytes

32000